<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 6. Dashboards</h2></div>

So far in this tutorial, we have seen how to generate plots with `.plot` or `.hvplot`, how to compose these plots together into layouts and overlays, how to link selections between these plots, and how to control visualizations with Panel widgets using `.interactive`. In this notebook, we will learn how to put all these pieces together to display (and serve) these components in a dashboard using Panel.

## Panel `pane` objects

So far we have only seen Panel used as a source of widgets, but Panel also offers [pane](https://panel.holoviz.org/reference/index.html#panes) objects that can display various types of data (including output from just about any plotting library). First let's import Panel and load the extension:

In [ ]:
import pathlib
import pandas as pd
import panel as pn
import xarray as xr
import holoviews as hv

pn.extension('tabulator', template='material', sizing_mode='stretch_width')

import colorcet as cc
import hvplot.xarray # noqa
import hvplot.pandas # noqa

Here, we have enabled some optional functionality from Panel, specifically the `tabulator` extension, and selected a default template controlling the overall look and feel of the final app. We also configured how plots should be sized by default (stretching to fit the width available). We'll come back to the idea of a template later. Here, let's look at a simple pane, e.g. a [Markdown pane](https://panel.holoviz.org/reference/panes/Markdown.html) that displays Markdown-format text:

In [ ]:
pn.pane.Markdown('## Earthquake Dashboard')

In [ ]:
logo_path = pathlib.Path('../assets/usgs_logo.png')

The `PNG` pane can display PNG images:

In [ ]:
pn.pane.PNG(logo_path, height=130)

## Using `pn.panel`

Instead of having to select the pane type explicitly, you can use the `pn.panel` function that tries to guess the appropriate representation given the input. For instance, here we generate the same two panels using `pn.panel` and grab handles on the resulting objects:

In [ ]:
dashboard_title = pn.panel('## Earthquakes')
usgs_logo = pn.panel(logo_path, height=130)

#### Exercise

Confirm that these two objects are of type `Markdown` and `PNG` respectively by using the `type` built-in. Explore using different markdown syntax such as *italic*, **bold** or adding bullet points. Finally, try displaying your own PNG image with a `PNG` pane, using either a local filename or URL.

## `Panel` objects

In addition to `pane` objects, Panel offers containers of type `panel` which allow you to position your components into various layouts. For instance, we can put a small version of our title and logo into a Panel `Row` layout:

In [ ]:
header = pn.Row(dashboard_title, pn.pane.PNG(logo_path, height=40))
header

Next let us load the earthquake dataset and make a basic plot of the sort we have seen earlier on in the tutorial:

In [ ]:
%%time
df = pd.read_parquet(pathlib.Path('../data/earthquakes-projected.parq'))
df = df.set_index('time').tz_convert(None).reset_index()
small_df = df.sample(frac=.01)

In [ ]:
%%time
ds = xr.open_dataarray(pathlib.Path('../data/raster/gpw_v4_population_density_rev11_2010_2pt5_min.nc'))
cleaned_ds = ds.where(ds.values != ds.nodatavals).sel(band=1)
cleaned_ds.name = 'population'

In [ ]:
sample_points = small_df.hvplot.points(x='longitude', y='latitude', c='mag', cmap=cc.CET_L4, responsive=True)
rasterized_pop = cleaned_ds.hvplot.image(rasterize=True, logz=True, clim=(1, None), responsive=True, min_height=400).opts(bgcolor='black')
earthquake_example = rasterized_pop * sample_points

Now we can combine this plot with our header in a `pn.Column`:

In [ ]:
mini_dashboard = pn.Column(header, earthquake_example)
mini_dashboard

## Showing and serving dashboards

Now`mini_dashboard` is a Panel object that can be displayed or served as a simple dashboard. To view the dashboard in a new tab, you can simply call `.show()`:

In [ ]:
# mini_dashboard.show()

If instead of `.show()` you use the `.servable()` method, you can serve the dashboard from this notebook using the command:

```bash
$ panel serve 06_Dashboard.ipynb
```

We will use this to serve a more sophisticated dashboard built in this notebook.

## A visual earthquake filter

From the last notebook, first subset the data:

In [ ]:
WEB_MERCATOR_LIMITS = (-20037508.342789244, 20037508.342789244)

subset_df = df[
    (df['northing']  < WEB_MERCATOR_LIMITS[1]) &
    (df['mag']       > 4) &
    (df['time']     >= pd.Timestamp('2017-01-01')) &
    (df['time']     <= pd.Timestamp('2018-01-01'))]

Declare the panel widgets:

In [ ]:
date_subrange = pn.widgets.DateRangeSlider(name='Date', 
                                        start=subset_df.time.iloc[0], 
                                        end=subset_df.time.iloc[-1])
mag_subrange = pn.widgets.FloatSlider(name='Magnitude', start=3, end=9, value=3)

Create an interactive `DataFrame` and use `hvplot` to generate a visualization of earthquakes plotted on a map and controlled with widgets:

In [ ]:
subset_dfi = subset_df.interactive(sizing_mode='stretch_width')
filtered_subrange = subset_dfi[
    (subset_dfi['mag']   > mag_subrange) &
    (subset_dfi['time'] >= date_subrange.param.value_start) &
    (subset_dfi['time'] <= date_subrange.param.value_end)]

geo = filtered_subrange.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500, tiles='ESRI')

This `geo` object works in Panel layouts, which means we can now add a header to it:

In [ ]:
pn.Column(header, geo)

As before, we now have a functional dashboard that we can show with `.show()` or serve with `.servable()`.

## Final Dashboard

We can now put all the concepts we have learned together to make a more sophisticated, interactive dashboard supporting linked selections:

In [ ]:
pn.state.template.sidebar_width = 250
pn.state.template.title = 'Earthquake Interactive Demo'

ls = hv.link_selections.instance(unselected_alpha=0.02)

# Table is not yet dynamically linked to the linked selection
table = filtered_subrange.pipe(ls.filter, selection_expr=ls.param.selection_expr)[['time', 'place', 'mag', 'depth']].pipe(
    pn.widgets.Tabulator, pagination='remote', page_size=15)

mag_hist = filtered_subrange.hvplot(
    y='mag', kind='hist', responsive=True, min_height=300)

depth_hist = filtered_subrange.hvplot(
    y='depth', kind='hist', responsive=True, min_height=300)

geo = filtered_subrange.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500,
    data_aspect=1, framewise=True, clim=(4, 10), line_color='black'
)

column = pn.Column(
    pn.Row(
        hv.element.tiles.ESRI() * ls(geo.holoviews()),
        table.panel()
    ),
    pn.Row(
        ls(depth_hist.holoviews()),
        ls(mag_hist.holoviews()),
    )
)

In [ ]:
filtered_subrange.widgets().servable(area='sidebar')

In [ ]:
column.servable(title='Earthquake Interactive Demo')

This dashboard should work in the notebook interface (with widgets shown separately from the plots above) for debugging, but a complex layout like that is meant to be served separately, e.g. using `panel serve --port 5067 06_Dashboards.ipynb` run in this directory. If working with the `anaconda-project` version of this tutorial, you can run:

```bash
anaconda-project run dashboard
```

Note that the code above can instead be pasted into a Python text file and run with `panel serve file.py`; none of the machinery depends on Jupyter or on being stored in the notebook format, and apps can be developed fully in text editors if preferred.

## Sharing

Once you are happy with the dashboard you have built, it's time to share it! You can find on [Panel's website](https://panel.holoviz.org/how_to/index.html#share-your-work) more details about how to:

- configure the server
- deploy the application on various platforms (e.g. Microsoft Azure, Google Cloud Platform, Hugging Face)
- save a dashboard as a static HTML file
- deploy a dashboard leveraging a new technology called WebAssembly thanks to which (and to Pyodide or PyScript) a Python application can run entirely in the browser, making it a lot easier to deploy a dashboard as you no longer need to bother setting up a server

Though this feature is brand new as of 6/2023 and not yet documented on Panel's website, you can now easily deploy Panel dashboards on [Anaconda Notebooks](https://anaconda.cloud/anaconda-tools) as well.

## Lumen

Panel is a highly flexible library to build apps and dashboards that provides all the required building blocks for you to assemble as you like. The [Lumen library](https://lumen.holoviz.org/) is a relatively new addition to the HoloViz ecosystem that is much more opinionated about what a data app is, handling many of the tasks involved automatically. Building on top of [Intake](https://intake.readthedocs.io/en/latest/index.html) Panel, Lumen can take care of ingesting, filtering, transforming and viewing data, all through a declarative specification that can be stored in YAML files.

## Conclusion

The techniques above and in the previous notebooks should enable you to build complex, deeply interlinked layouts and dashboards that help you understand your data and uncover hidden relationships, with custom interactivity for exploring any parameters of interest. You can explore further at the websites for any of the tools mentioned here, starting at [hvplot.holoviz.org](https://hvplot.holoviz.org) and moving on from there as needed.

In the tutorials so far, we have focused almost exclusively on the highest-level APIs provided by HoloViz, namely `.hvplot()` and `.interactive()`. These interfaces let you focus on the data you are trying to work with, without getting bogged down in writing dozens or hundreds of lines of plotting or callback code. Of course, they don't cover every possible type of plot or interactivity, and if you want full control, you'll need to learn the lower-level APIs provided by HoloViz tools (HoloViews and Panel), or the even lower-level tools, like Bokeh. The remaining tutorials are entirely optional, but they introduce some of those lower-level APIs so that you can see how to do things more manually when needed. These later tutorials are particularly useful for understanding the examples of interesting apps at [examples.holoviz.org](https://examples.holoviz.org), most of which were written before the simple `.interactive()` interface was developed.